In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
import gc
from os import path
from sklearn.preprocessing import LabelEncoder
from pandas.core.dtypes.dtypes import CategoricalDtype
from tqdm import tqdm
from datetime import date 
import holidays
import lightgbm as lgb


warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

# label encoding
le = LabelEncoder()

In [2]:
class ConvertToDatetime(TransformerMixin):
        
    def transform(self, df, **transform_params):
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [3]:
file_dtype = {
    'train': {'building_id': np.int16, 'meter': np.int8, 'meter_reading': np.float32},
    'test': {'building_id': np.int16, 'meter': np.int8},
    'building_metadata': {'site_id': np.int8, 'building_id': np.uint16, 'square_feet': np.float32, 'year_built': np.float16, 'floor_count': np.float16},
    'weather_test' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16},
    'weather_train' : {'site_id': np.int8, 'air_temperature': np.float16, 'cloud_coverage': np.float16, 'dew_temperature': np.float16,
                     'precip_depth_1_hr': np.float16, 'sea_level_pressure': np.float16, 'wind_direction': np.float16, 'wind_speed': np.float16}
}

def loadFile(name):
    for dir_path in ['../input/ashrae-energy-prediction/','../input/_ashrae-energy-prediction/']:
        if path.exists(dir_path + name + '.csv'):
            return  ConvertToDatetime().transform(
                pd.read_csv(dir_path + name + '.csv', dtype=file_dtype[name]))
        


In [4]:
building = loadFile('building_metadata')
weather_train = loadFile('weather_train')
weather_test = loadFile('weather_test')
train = loadFile('train')
test = loadFile('test')



In [5]:
# align weather
def weatherSiteOffsets():
    weather = pd.concat([weather_train,weather_test],ignore_index=True)
    weather['timestamp'] = pd.to_datetime(weather['timestamp'])
    weather_key = ['site_id', 'timestamp']

    temp_skeleton = weather[weather_key + ['air_temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

    # calculate ranks of hourly temperatures within date/site_id chunks
    temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.date])['air_temperature'].rank('average')
    
    # create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
    df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

    # Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
    site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
    site_ids_offsets.index.name = 'site_id'
    return site_ids_offsets

site_time_offsets_df = weatherSiteOffsets()

def alignWeather(df):
        df['offset'] = df.site_id.map(site_time_offsets_df)
        df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
        df['timestamp'] = df['timestamp_aligned']
        del df['timestamp_aligned'], df['offset']
        gc.collect()
        return df

weather_train = alignWeather(weather_train)
weather_test = alignWeather(weather_test)

del site_time_offsets_df

In [6]:
# See holiday notebook to generate
if path.exists('../input/holiday-pickle/holiday_df.pickle'):
    holiday_df = pd.read_pickle('../input/holiday-pickle/holiday_df.pickle')
else:
    holiday_df = pd.read_pickle('../input/ashrae-energy-prediction/holiday_df.pickle')

In [7]:
# "As you can see above, this data looks weired until May 20. It is 
# reported in this discussion by @barnwellguy that All electricity
# meter is 0 until May 20 for site_id == 0. Let's remove these data 
# from training data."
# https://www.kaggle.com/kaushal2896/ashrae-eda-fe-lightgbm-1-13
class RmS0M0(TransformerMixin):
  
    def transform(self, df, **transform_params):
        return df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

    def fit(self, X, y=None, **fit_params):
        return self
    


In [8]:
class ImputeWeather(TransformerMixin):

    def __init__(self, method:str='linear', gap_limit:int=None, limit_direction:str='forward'):
        self._method = method
        self._gap_limit = gap_limit
        self._limit_direction = limit_direction
        
    def transform(self, weather_df, **transform_params):
        grouped_weather_df = weather_df.groupby('site_id').apply(lambda group: group.interpolate(method=self._method, limit=self._gap_limit, limit_direction=self._limit_direction))
        if 'cloud_coverage' in grouped_weather_df.columns:
            grouped_weather_df['cloud_coverage'] = grouped_weather_df['cloud_coverage'].round(decimals=0).clip(0,8)
        grouped_weather_df.reset_index(inplace=True)
        weather_df = grouped_weather_df.drop(['index'], axis=1)
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
print(weather_train.head(20))
print(ImputeWeather().transform(weather_train.head(20)))

    site_id           timestamp  air_temperature  cloud_coverage  \
0         0 2015-12-31 19:00:00        25.000000             6.0   
1         0 2015-12-31 20:00:00        24.406250             NaN   
2         0 2015-12-31 21:00:00        22.796875             2.0   
3         0 2015-12-31 22:00:00        21.093750             2.0   
4         0 2015-12-31 23:00:00        20.000000             2.0   
5         0 2016-01-01 00:00:00        19.406250             NaN   
6         0 2016-01-01 01:00:00        21.093750             6.0   
7         0 2016-01-01 02:00:00        21.093750             NaN   
8         0 2016-01-01 03:00:00        20.593750             NaN   
9         0 2016-01-01 04:00:00        21.093750             NaN   
10        0 2016-01-01 05:00:00        21.093750             NaN   
11        0 2016-01-01 06:00:00        20.593750             NaN   
12        0 2016-01-01 07:00:00        18.906250             6.0   
13        0 2016-01-01 08:00:00        20.000000

In [9]:
class AddWeatherLags(TransformerMixin):
    
    def __init__(self, window):
        self._window = window
        
    def transform(self, weather_df, **transform_params):
        group_df = weather_df.groupby(['site_id'])
        cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
        rolled = group_df[cols].rolling(window=self._window, min_periods=0)
        lag_mean = rolled.mean().reset_index().astype(np.float16)
        lag_max = rolled.max().reset_index().astype(np.float16)
        lag_min = rolled.min().reset_index().astype(np.float16)
        lag_std = rolled.std().reset_index().astype(np.float16)
        for col in cols:
            weather_df[f'{col}_mean_lag{self._window}'] = lag_mean[col]
            weather_df[f'{col}_max_lag{self._window}'] = lag_max[col]
            weather_df[f'{col}_min_lag{self._window}'] = lag_min[col]
            weather_df[f'{col}_std_lag{self._window}'] = lag_std[col]
        del group_df, rolled
        gc.collect()
        return weather_df

    def fit(self, X, y=None, **fit_params):
        return self
    
print(AddWeatherLags(72).transform(weather_train.head(20)))

    site_id           timestamp  air_temperature  cloud_coverage  \
0         0 2015-12-31 19:00:00        25.000000             6.0   
1         0 2015-12-31 20:00:00        24.406250             NaN   
2         0 2015-12-31 21:00:00        22.796875             2.0   
3         0 2015-12-31 22:00:00        21.093750             2.0   
4         0 2015-12-31 23:00:00        20.000000             2.0   
5         0 2016-01-01 00:00:00        19.406250             NaN   
6         0 2016-01-01 01:00:00        21.093750             6.0   
7         0 2016-01-01 02:00:00        21.093750             NaN   
8         0 2016-01-01 03:00:00        20.593750             NaN   
9         0 2016-01-01 04:00:00        21.093750             NaN   
10        0 2016-01-01 05:00:00        21.093750             NaN   
11        0 2016-01-01 06:00:00        20.593750             NaN   
12        0 2016-01-01 07:00:00        18.906250             6.0   
13        0 2016-01-01 08:00:00        20.000000

[20 rows x 37 columns]


In [10]:
class AddBuilding(TransformerMixin):

    def __init__(self, building_df):
        self._b_df = building_df
        
    def transform(self, df, **transform_params):
        return df.merge(_b_df, on='building_id', how='left')

    def fit(self, X, y=None, **fit_params):
        return self

In [11]:
    
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114483#latest-660771
# https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114874#latest-660970
class AddHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
        return df

    def fit(self, X, y=None, **fit_params):
        return self
# Test 
addHolidays = AddHolidays()
print(addHolidays.transform(train.head(2000).merge(building, on='building_id', how='left'))[['building_id','timestamp','holiday']])

      building_id  timestamp         holiday
0               0 2016-01-01  New Year's Day
1               1 2016-01-01  New Year's Day
2               2 2016-01-01  New Year's Day
3               3 2016-01-01  New Year's Day
4               4 2016-01-01  New Year's Day
5               5 2016-01-01  New Year's Day
6               6 2016-01-01  New Year's Day
7               7 2016-01-01  New Year's Day
8               8 2016-01-01  New Year's Day
9               9 2016-01-01  New Year's Day
10             10 2016-01-01  New Year's Day
11             11 2016-01-01  New Year's Day
12             12 2016-01-01  New Year's Day
13             13 2016-01-01  New Year's Day
14             14 2016-01-01  New Year's Day
15             15 2016-01-01  New Year's Day
16             16 2016-01-01  New Year's Day
17             17 2016-01-01  New Year's Day
18             18 2016-01-01  New Year's Day
19             19 2016-01-01  New Year's Day
20             20 2016-01-01  New Year's Day
21        

In [12]:
class RmHolidays(TransformerMixin):
    def transform(self, df, **transform_params):
        df = df.merge(holiday_df, on=['building_id','meter','timestamp','site_id'], how='left')
        df = df.drop(df[df['holiday'].notnull()].index)
        df = df.drop(['holiday'], axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

# Test you should see the new years removed
rmHolidays = RmHolidays()
print(rmHolidays.transform(train.head(100000).merge(building, on='building_id', how='left')))

       building_id  meter           timestamp  meter_reading  site_id  \
55121            0      0 2016-01-02 00:00:00       0.000000        0   
55122            1      0 2016-01-02 00:00:00       0.000000        0   
55123            2      0 2016-01-02 00:00:00       0.000000        0   
55124            3      0 2016-01-02 00:00:00       0.000000        0   
55125            4      0 2016-01-02 00:00:00       0.000000        0   
55126            5      0 2016-01-02 00:00:00       0.000000        0   
55127            6      0 2016-01-02 00:00:00       0.000000        0   
55128            7      0 2016-01-02 00:00:00       0.000000        0   
55129            8      0 2016-01-02 00:00:00       0.000000        0   
55130            9      0 2016-01-02 00:00:00       0.000000        0   
55131           10      0 2016-01-02 00:00:00       0.000000        0   
55132           11      0 2016-01-02 00:00:00       0.000000        0   
55133           12      0 2016-01-02 00:00:00      

[41839 rows x 9 columns]


In [13]:
class LogSquareFeet(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['log_square_feet'] = np.float16(np.log(df['square_feet']))
        return df

    def fit(self, X, y=None, **fit_params):
        return self
print(building.head(20)['square_feet'])

0       7432.0
1       2720.0
2       5376.0
3      23685.0
4     116607.0
5       8000.0
6      27926.0
7     121074.0
8      60809.0
9      27000.0
10    370773.0
11     49073.0
12     37100.0
13     99380.0
14     86250.0
15     83957.0
16     54644.0
17     15250.0
18    111891.0
19     18717.0
Name: square_feet, dtype: float32


In [14]:
class SetCatTypes(TransformerMixin):
        
    def transform(self, df, **transform_params):
        df['primary_use']= df['primary_use'].astype('category')
        df['meter'] = df["meter"].astype('category')
        df['site_id'] = df["site_id"].astype('category')
        df['building_id'] = df['building_id'].astype('category')
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [15]:
class ImputeCloudCoverage(TransformerMixin):
        
    def transform(self, df, **transform_params):
        # set age of building to mediam of site_id
        # else if set ot overall median
        median = df['cloud_coverage'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['cloud_coverage'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = i_median
            else:
                df.loc[(df['cloud_coverage'].isnull()) & (df['site_id'] == i), 'cloud_coverage'] = median
        df['cloud_coverage'] = np.uint8(df['cloud_coverage'])
        df['cloud_coverage'] = df['cloud_coverage']
        del median
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [16]:
class CloudTimeCat(TransformerMixin):
        
    def transform(self, df, **transform_params):
        tempDf = df[['cloud_coverage', 'hour']].astype('int')
        tempDf['cloud_coverage'] = (tempDf['cloud_coverage']).astype('int')
        tempDf['hour'] = (tempDf['hour']).astype('int')
        tempDf = tempDf.astype('str')
        df['cloud_time_cat'] = 'c' + tempDf['cloud_coverage'] + 't' + tempDf['hour']
        df['cloud_time_cat'] = df['cloud_time_cat'].astype('category')
        del tempDf
        gc.collect()
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self


In [17]:
class DropCols(TransformerMixin):

    def __init__(self, drop_cols):
        self._drop_cols = drop_cols
        
    def transform(self, df, **transform_params):
        df = df.drop(self._drop_cols, axis=1)
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [18]:
class ImputeYearBuilt(TransformerMixin):

    def transform(self, df, **transform_params):
        year_built_median = df['year_built'].median()
        # Set all year_built NaNs to site mean for year_built
        for i, i_median in df.groupby(['site_id'])['year_built'].median().items():
            if not np.isnan(i_median):
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = i_median
            else:
                df.loc[(df['year_built'].isnull()) & (df['site_id'] == i), 'year_built'] = year_built_median
        df['building_age'] = np.uint8(df['year_built']-1900)
        del year_built_median
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [19]:
class AddMeterDummies(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        for i in range(4):
            df['_meter_'+str(i)] = (df['building_id'].isin(
                train.loc[train['meter'] == i].building_id.unique()))
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [20]:
class AddTimeFeatures(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        df['dayofweek'] = df['timestamp'].dt.dayofweek.astype('category') # vs weekend?
        #df['weekday'] = df['timestamp'].dt.weekday.astype('category')
        #df['dayofweek_hour'] = (df['timestamp'].dt.dayofweek * 24) + df['timestamp'].dt.hour
        #df['dayofweek_hour'] = df['dayofweek_hour'].astype('category')
        #df['week'] = df['timestamp'].dt.week.astype('category')
        df['hour'] = df['timestamp'].dt.hour.astype('category')
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [21]:
class AddRelativeHumidity(TransformerMixin):
        
    def transform(self, df_a, **transform_params):
        df = df_a
        # code here
        return df
        
    def fit(self, X, y=None, **fit_params):
        return self

In [22]:
class FillMean(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].mean())
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [23]:
class FillZeros(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(0)
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [24]:
class FillMedian(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].median())
        return df

    def fit(self, X, y=None, **fit_params):
        return self


In [25]:
class FillPopular(TransformerMixin):

    def __init__(self, cols):
        self._cols = cols
        
    def transform(self, df, **transform_params):
        for col in self._cols:
            df[col] = df[col].fillna(df[col].value_counts()[0])
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [26]:
class MarkNaNs(TransformerMixin):
        
    def transform(self, df, **transform_params):
        for col in  df.columns[df.isna().any()].tolist():
            df['_' + col + '_nan' ] = df[col].isnull()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [27]:
class GC(TransformerMixin):
        
    def transform(self, df, **transform_params):
        gc.collect()
        return df

    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
# declare model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_squared_error
from lightgbm import LGBMRegressor


def rmsle(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(y, y_pred.clip(0)))

def rmse(y, y_pred):
    # hack to prevent negative numbers
    return mean_squared_error(y, y_pred.clip(0))

def rmsee(y, y_pred):
    # hack to prevent negative numbers
    return np.sqrt(mean_squared_log_error(np.expm1(y.clip(0)), np.expm1(y_pred.clip(0))))
    
rmsle_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmse_scorer = make_scorer(
    lambda y_true, y_pred : rmsle(y_true, y_pred), 
    greater_is_better=False)

rmsee_scorer = make_scorer(
    lambda y_true, y_pred : rmsee(y_true, y_pred), 
    greater_is_better=False)

def lbm_rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

# rob's custome function to do RMSLE while in the log1p space
def lbm_rmslee(y_true, y_pred):
    return 'RMSLEE', np.sqrt(np.mean(np.power(y_pred - y_true, 2))), False



In [29]:
weather_pipes = Pipeline(
    steps=[
        #('convertToDatetime', ConvertToDatetime()),
        ('imputeWeather', ImputeWeather()),
        ('fillMean',FillMean(['air_temperature','dew_temperature'
                              , 'precip_depth_1_hr', 'sea_level_pressure'])),
        ('imputeCloudCoverage', ImputeCloudCoverage()),
        ('addWeatherLags3', AddWeatherLags(3)),
        ('addWeatherLags72', AddWeatherLags(72)),
    ]
)

building_pipes = Pipeline(
    steps=[
        ('logSquareFeet', LogSquareFeet()),
        ('imputeYearBuilt', ImputeYearBuilt()),
        ('fillMean',FillMean(['floor_count'])),
        ('dropClos', DropCols(['square_feet', 'year_built'])),
    ]
)


# pre_a_pipes is for preprocessing that doesn't change impute
# values
x_pipes = Pipeline(
    steps=[
        #('markNans',MarkNaNs()),
        #('convertToDatetime', ConvertToDatetime()),
        ('rmS0M0', RmS0M0()),
        #('addHolidays', AddHolidays()),
        #('rmHolidays', RmHolidays()), called manually in fold
        ('addRelativeHumidity',AddRelativeHumidity()),
        ('addTimeFeatures', AddTimeFeatures()),
        ('setCatTypes', SetCatTypes()),
        ('fillMean',FillMean([])),
        ('fillZeros',FillZeros([])),
        ('dropClos', DropCols(['timestamp'])),
        ('GC', GC())
    ]
)

In [30]:
def getOutsideFoldXY(train_index):
    X = train.iloc[train_index].drop('meter_reading', axis=1)
    X_buildings = building[building['building_id'].isin(X['building_id'].unique())]
    X_weather = building[building['building_id'].isin(X['building_id'].unique())]
    X = x_pipes.transform(
        X
            .merge(building_pipes.transform(X_buildings), on='building_id', how='left')
            .merge(weather_train_trans, on=['site_id', 'timestamp'], how='left')
        )
    f_train_y = np.log1p(train.iloc[train_index]['meter_reading'])
    print(X.columns)
    return X,f_train_y



def getInFoldXY(train_index):
    X = train.iloc[train_index]
    X_buildings = building[building['building_id'].isin(X['building_id'].unique())]
    X = X.merge(building_pipes.transform(X_buildings), on='building_id', how='left')
    X_weather = weather_train[
        (weather_train['site_id'].isin(X['site_id'].unique())) 
         & (weather_train['timestamp'].isin(X['timestamp'].unique())) 
    ]
    X = x_pipes.transform(
        rmHolidays.transform(
            X.merge(weather_pipes.transform(X_weather), how='left')))
    return X

sample_train_X = getInFoldXY(train.sample(n=20,  random_state=42).index)
print(sample_train_X)

   building_id meter  meter_reading site_id                    primary_use  \
0         1324     1       0.000000      14  Entertainment/public assembly   
1         1013     0      32.000099      10                      Education   
2          229     1     567.655029       2                      Education   
3          217     3       0.000000       2                      Education   
4         1434     0      65.750000      15                      Education   
5         1047     0      90.983299      12                Public services   
6          911     1     295.063995       9                      Education   
7         1039     0      16.900000      12                      Education   
8          265     0     128.369995       2                         Office   
9          896     0     300.000000       9                      Education   
10         973     0     247.000000       9                         Office   
11         813     0      10.958300       8                Publi

In [31]:
# model fit params
gbm_params = {
    'n_estimators' : 500, # for accuracy use large numbers like 6000 
    'learning_rate': 0.1,
    'feature_fraction' : 0.9,
    'bagging_freq': 5,
    'bagging_fraction': 0.1,
    'subsample' : 0.1,  # 
    'subsample_freq' : 1,
    'num_leaves' : 20,
    'max_depth' : 9,
    'metric':'rmse',
    'lambda_l1' : 1,  
    'lambda_l2': 1,
    'verbose': 100
}

In [33]:
## cross val mdoels



# this stratified strategy from
# https://www.kaggle.com/isaienkov/lightgbm-fe-1-19/notebook
folds = 5
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

def cvTrainEnsemble(train, gbm_params):
    meter_models = [ [], [], [], [] ]
    for train_index, val_index in kf.split(train, train['building_id']):
        f_train = getInFoldXY(train_index)
        f_val = getInFoldXY(val_index)
        for i in range(4):
            f_train_m = f_train[f_train['meter'] == i]
            f_val_m = f_val[f_val['meter'] == i]
            gbm_params_m = gbm_params
            if i == 0:
                gbm_params_m['n_estimators']=500
            elif i == 1:
                gbm_params_m['learning_rate']=0.05
                gbm_params_m['bagging_fraction']=0.05
            elif i == 2:
                gbm_params_m['learning_rate']=0.05
                gbm_params_m['bagging_fraction']=0.08
            else:
                gbm_params_m['learning_rate']=0.03
                gbm_params_m['bagging_fraction']=0.09                
            gbm = LGBMRegressor(**gbm_params_m)
            gbm.fit(f_train_m.drop('meter_reading', axis=1), np.log1p(f_train_m['meter_reading']),
                eval_set=[(f_val_m.drop('meter_reading', axis=1), np.log1p(f_val_m['meter_reading']))],
                # https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114722#latest-660848
                # eval_metric=lbm_rmslee,
                early_stopping_rounds=20)
            meter_models[i].append(gbm)
            del f_train_m, f_val_m, gbm
            gc.collect()
        del f_train, f_val
        gc.collect()
    return meter_models


In [ ]:
%%time
meter_models = cvTrainEnsemble(train, gbm_params)

[1]	valid_0's rmse: 1.46621
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 1.36041
[3]	valid_0's rmse: 1.26609
[4]	valid_0's rmse: 1.18325
[5]	valid_0's rmse: 1.11098
[6]	valid_0's rmse: 1.05872
[7]	valid_0's rmse: 1.00196
[8]	valid_0's rmse: 0.952217
[9]	valid_0's rmse: 0.908818
[10]	valid_0's rmse: 0.871351
[11]	valid_0's rmse: 0.846817
[12]	valid_0's rmse: 0.816523
[13]	valid_0's rmse: 0.790894
[14]	valid_0's rmse: 0.768847
[15]	valid_0's rmse: 0.750301
[16]	valid_0's rmse: 0.734739
[17]	valid_0's rmse: 0.720429
[18]	valid_0's rmse: 0.708115
[19]	valid_0's rmse: 0.697256
[20]	valid_0's rmse: 0.687343
[21]	valid_0's rmse: 0.679643
[22]	valid_0's rmse: 0.67141
[23]	valid_0's rmse: 0.664146
[24]	valid_0's rmse: 0.658155
[25]	valid_0's rmse: 0.652445
[26]	valid_0's rmse: 0.647583
[27]	valid_0's rmse: 0.643188
[28]	valid_0's rmse: 0.639107
[29]	valid_0's rmse: 0.634173
[30]	valid_0's rmse: 0.630941
[31]	valid_0's rmse: 0.626381
[32]	valid_0's rmse: 0.622

[267]	valid_0's rmse: 0.46046
[268]	valid_0's rmse: 0.459817
[269]	valid_0's rmse: 0.459547
[270]	valid_0's rmse: 0.45948
[271]	valid_0's rmse: 0.459239
[272]	valid_0's rmse: 0.459159
[273]	valid_0's rmse: 0.459037
[274]	valid_0's rmse: 0.458889
[275]	valid_0's rmse: 0.458728
[276]	valid_0's rmse: 0.458283
[277]	valid_0's rmse: 0.458071
[278]	valid_0's rmse: 0.45795
[279]	valid_0's rmse: 0.457779
[280]	valid_0's rmse: 0.457614
[281]	valid_0's rmse: 0.457017
[282]	valid_0's rmse: 0.456862
[283]	valid_0's rmse: 0.456542
[284]	valid_0's rmse: 0.456446
[285]	valid_0's rmse: 0.45634
[286]	valid_0's rmse: 0.456204
[287]	valid_0's rmse: 0.456071
[288]	valid_0's rmse: 0.455962
[289]	valid_0's rmse: 0.455773
[290]	valid_0's rmse: 0.455548
[291]	valid_0's rmse: 0.455353
[292]	valid_0's rmse: 0.455215
[293]	valid_0's rmse: 0.455028
[294]	valid_0's rmse: 0.454914
[295]	valid_0's rmse: 0.454652
[296]	valid_0's rmse: 0.454539
[297]	valid_0's rmse: 0.454296
[298]	valid_0's rmse: 0.454053
[299]	valid_

[30]	valid_0's rmse: 1.4171
[31]	valid_0's rmse: 1.40516
[32]	valid_0's rmse: 1.39364
[33]	valid_0's rmse: 1.38641
[34]	valid_0's rmse: 1.3797
[35]	valid_0's rmse: 1.37071
[36]	valid_0's rmse: 1.36096
[37]	valid_0's rmse: 1.35166
[38]	valid_0's rmse: 1.34462
[39]	valid_0's rmse: 1.33705
[40]	valid_0's rmse: 1.32901
[41]	valid_0's rmse: 1.32204
[42]	valid_0's rmse: 1.31581
[43]	valid_0's rmse: 1.31011
[44]	valid_0's rmse: 1.30335
[45]	valid_0's rmse: 1.29844
[46]	valid_0's rmse: 1.29284
[47]	valid_0's rmse: 1.28756
[48]	valid_0's rmse: 1.28166
[49]	valid_0's rmse: 1.27729
[50]	valid_0's rmse: 1.27159
[51]	valid_0's rmse: 1.26661
[52]	valid_0's rmse: 1.26297
[53]	valid_0's rmse: 1.26031
[54]	valid_0's rmse: 1.25632
[55]	valid_0's rmse: 1.25282
[56]	valid_0's rmse: 1.24937
[57]	valid_0's rmse: 1.24638
[58]	valid_0's rmse: 1.24258
[59]	valid_0's rmse: 1.23909
[60]	valid_0's rmse: 1.23541
[61]	valid_0's rmse: 1.23263
[62]	valid_0's rmse: 1.22972
[63]	valid_0's rmse: 1.227
[64]	valid_0's rms

[306]	valid_0's rmse: 1.06174
[307]	valid_0's rmse: 1.06127
[308]	valid_0's rmse: 1.06096
[309]	valid_0's rmse: 1.0607
[310]	valid_0's rmse: 1.06042
[311]	valid_0's rmse: 1.06019
[312]	valid_0's rmse: 1.05999
[313]	valid_0's rmse: 1.0599
[314]	valid_0's rmse: 1.0598
[315]	valid_0's rmse: 1.05926
[316]	valid_0's rmse: 1.05908
[317]	valid_0's rmse: 1.05885
[318]	valid_0's rmse: 1.05874
[319]	valid_0's rmse: 1.05854
[320]	valid_0's rmse: 1.05827
[321]	valid_0's rmse: 1.05761
[322]	valid_0's rmse: 1.05705
[323]	valid_0's rmse: 1.0563
[324]	valid_0's rmse: 1.05572
[325]	valid_0's rmse: 1.05521
[326]	valid_0's rmse: 1.05494
[327]	valid_0's rmse: 1.05466
[328]	valid_0's rmse: 1.05416
[329]	valid_0's rmse: 1.05373
[330]	valid_0's rmse: 1.05348
[331]	valid_0's rmse: 1.05329
[332]	valid_0's rmse: 1.05313
[333]	valid_0's rmse: 1.05291
[334]	valid_0's rmse: 1.05249
[335]	valid_0's rmse: 1.05227
[336]	valid_0's rmse: 1.05205
[337]	valid_0's rmse: 1.0518
[338]	valid_0's rmse: 1.05115
[339]	valid_0's

[78]	valid_0's rmse: 1.30906
[79]	valid_0's rmse: 1.30672
[80]	valid_0's rmse: 1.30505
[81]	valid_0's rmse: 1.30292
[82]	valid_0's rmse: 1.30112
[83]	valid_0's rmse: 1.29918
[84]	valid_0's rmse: 1.29789
[85]	valid_0's rmse: 1.29668
[86]	valid_0's rmse: 1.29443
[87]	valid_0's rmse: 1.29239
[88]	valid_0's rmse: 1.29104
[89]	valid_0's rmse: 1.28852
[90]	valid_0's rmse: 1.28689
[91]	valid_0's rmse: 1.28567
[92]	valid_0's rmse: 1.28448
[93]	valid_0's rmse: 1.28362
[94]	valid_0's rmse: 1.28221
[95]	valid_0's rmse: 1.28097
[96]	valid_0's rmse: 1.27905
[97]	valid_0's rmse: 1.27745
[98]	valid_0's rmse: 1.27627
[99]	valid_0's rmse: 1.27548
[100]	valid_0's rmse: 1.27419
[101]	valid_0's rmse: 1.27292
[102]	valid_0's rmse: 1.27186
[103]	valid_0's rmse: 1.27079
[104]	valid_0's rmse: 1.27004
[105]	valid_0's rmse: 1.26888
[106]	valid_0's rmse: 1.26748
[107]	valid_0's rmse: 1.26637
[108]	valid_0's rmse: 1.26573
[109]	valid_0's rmse: 1.26499
[110]	valid_0's rmse: 1.2631
[111]	valid_0's rmse: 1.26205
[11

...

[200]	valid_0's rmse: 1.09898
[201]	valid_0's rmse: 1.09816
[202]	valid_0's rmse: 1.09787
[203]	valid_0's rmse: 1.09769
[204]	valid_0's rmse: 1.09741
[205]	valid_0's rmse: 1.09714
[206]	valid_0's rmse: 1.09648
[207]	valid_0's rmse: 1.09541
[208]	valid_0's rmse: 1.09496
[209]	valid_0's rmse: 1.09454
[210]	valid_0's rmse: 1.09411
[211]	valid_0's rmse: 1.09354
[212]	valid_0's rmse: 1.09305
[213]	valid_0's rmse: 1.09234
[214]	valid_0's rmse: 1.09177
[215]	valid_0's rmse: 1.09149
[216]	valid_0's rmse: 1.09118
[217]	valid_0's rmse: 1.091
[218]	valid_0's rmse: 1.09081
[219]	valid_0's rmse: 1.0906
[220]	valid_0's rmse: 1.09036
[221]	valid_0's rmse: 1.09005
[222]	valid_0's rmse: 1.08977
[223]	valid_0's rmse: 1.08947
[224]	valid_0's rmse: 1.08924
[225]	valid_0's rmse: 1.08904
[226]	valid_0's rmse: 1.08876
[227]	valid_0's rmse: 1.08829
[228]	valid_0's rmse: 1.08805
[229]	valid_0's rmse: 1.08776
[230]	valid_0's rmse: 1.08749
[231]	valid_0's rmse: 1.08717
[232]	valid_0's rmse: 1.08672
[233]	valid_0

[474]	valid_0's rmse: 1.02487
[475]	valid_0's rmse: 1.02479
[476]	valid_0's rmse: 1.02475
[477]	valid_0's rmse: 1.02454
[478]	valid_0's rmse: 1.02432
[479]	valid_0's rmse: 1.02419
[480]	valid_0's rmse: 1.02414
[481]	valid_0's rmse: 1.02406
[482]	valid_0's rmse: 1.02398
[483]	valid_0's rmse: 1.02381
[484]	valid_0's rmse: 1.02372
[485]	valid_0's rmse: 1.02367
[486]	valid_0's rmse: 1.02329
[487]	valid_0's rmse: 1.02314
[488]	valid_0's rmse: 1.02285
[489]	valid_0's rmse: 1.0226
[490]	valid_0's rmse: 1.02246
[491]	valid_0's rmse: 1.0222
[492]	valid_0's rmse: 1.02206
[493]	valid_0's rmse: 1.02185
[494]	valid_0's rmse: 1.02176
[495]	valid_0's rmse: 1.02172
[496]	valid_0's rmse: 1.02148
[497]	valid_0's rmse: 1.0213
[498]	valid_0's rmse: 1.02115
[499]	valid_0's rmse: 1.02107
[500]	valid_0's rmse: 1.02088
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 1.02088
[1]	valid_0's rmse: 2.54728
Training until validation scores don't improve for 20 rounds
[2]	valid_0's rmse: 2.4660

[247]	valid_0's rmse: 1.18309
[248]	valid_0's rmse: 1.18293
[249]	valid_0's rmse: 1.18276
[250]	valid_0's rmse: 1.18262
[251]	valid_0's rmse: 1.18209
[252]	valid_0's rmse: 1.1817
[253]	valid_0's rmse: 1.18141
[254]	valid_0's rmse: 1.18098
[255]	valid_0's rmse: 1.18054
[256]	valid_0's rmse: 1.18028
[257]	valid_0's rmse: 1.18009
[258]	valid_0's rmse: 1.17992
[259]	valid_0's rmse: 1.17976
[260]	valid_0's rmse: 1.1795
[261]	valid_0's rmse: 1.17935
[262]	valid_0's rmse: 1.17913
[263]	valid_0's rmse: 1.17887
[264]	valid_0's rmse: 1.1785
[265]	valid_0's rmse: 1.17809
[266]	valid_0's rmse: 1.17782
[267]	valid_0's rmse: 1.17749
[268]	valid_0's rmse: 1.1771
[269]	valid_0's rmse: 1.1768
[270]	valid_0's rmse: 1.17656
[271]	valid_0's rmse: 1.17606
[272]	valid_0's rmse: 1.1756
[273]	valid_0's rmse: 1.17512
[274]	valid_0's rmse: 1.1747
[275]	valid_0's rmse: 1.17429
[276]	valid_0's rmse: 1.17393
[277]	valid_0's rmse: 1.17371
[278]	valid_0's rmse: 1.17343
[279]	valid_0's rmse: 1.17323
[280]	valid_0's r

[17]	valid_0's rmse: 1.98442
[18]	valid_0's rmse: 1.95901
[19]	valid_0's rmse: 1.93501
[20]	valid_0's rmse: 1.9117
[21]	valid_0's rmse: 1.8892
[22]	valid_0's rmse: 1.86829
[23]	valid_0's rmse: 1.84866
[24]	valid_0's rmse: 1.82924
[25]	valid_0's rmse: 1.81047
[26]	valid_0's rmse: 1.79273
[27]	valid_0's rmse: 1.77547
[28]	valid_0's rmse: 1.75944
[29]	valid_0's rmse: 1.7441
[30]	valid_0's rmse: 1.72927
[31]	valid_0's rmse: 1.71474
[32]	valid_0's rmse: 1.70091
[33]	valid_0's rmse: 1.69208
[34]	valid_0's rmse: 1.68371
[35]	valid_0's rmse: 1.67157
[36]	valid_0's rmse: 1.65964
[37]	valid_0's rmse: 1.64828
[38]	valid_0's rmse: 1.64144
[39]	valid_0's rmse: 1.63089
[40]	valid_0's rmse: 1.621
[41]	valid_0's rmse: 1.61123
[42]	valid_0's rmse: 1.60202
[43]	valid_0's rmse: 1.59327
[44]	valid_0's rmse: 1.58476
[45]	valid_0's rmse: 1.57958
[46]	valid_0's rmse: 1.5717
[47]	valid_0's rmse: 1.56432
[48]	valid_0's rmse: 1.55724
[49]	valid_0's rmse: 1.55036
[50]	valid_0's rmse: 1.54398
[51]	valid_0's rmse:

[294]	valid_0's rmse: 1.30468
[295]	valid_0's rmse: 1.30447
[296]	valid_0's rmse: 1.30419
[297]	valid_0's rmse: 1.30386
[298]	valid_0's rmse: 1.30367
[299]	valid_0's rmse: 1.30348
[300]	valid_0's rmse: 1.30334
[301]	valid_0's rmse: 1.30312
[302]	valid_0's rmse: 1.30272
[303]	valid_0's rmse: 1.30246
[304]	valid_0's rmse: 1.30226
[305]	valid_0's rmse: 1.30178
[306]	valid_0's rmse: 1.30157
[307]	valid_0's rmse: 1.30137
[308]	valid_0's rmse: 1.30105
[309]	valid_0's rmse: 1.30073
[310]	valid_0's rmse: 1.30045
[311]	valid_0's rmse: 1.30027
[312]	valid_0's rmse: 1.30008
[313]	valid_0's rmse: 1.29991
[314]	valid_0's rmse: 1.29967
[315]	valid_0's rmse: 1.29947
[316]	valid_0's rmse: 1.29935
[317]	valid_0's rmse: 1.29919
[318]	valid_0's rmse: 1.29905
[319]	valid_0's rmse: 1.29889
[320]	valid_0's rmse: 1.29873
[321]	valid_0's rmse: 1.29847
[322]	valid_0's rmse: 1.29826
[323]	valid_0's rmse: 1.2981
[324]	valid_0's rmse: 1.29802
[325]	valid_0's rmse: 1.2978
[326]	valid_0's rmse: 1.29764
[327]	valid_

In [ ]:
for i in range(4):
    print('meter: '+ str(i))
    for model in meter_models[i]:
        print(model.best_score_['valid_0']['rmse'])


In [ ]:
# Importance rank for first model in cross val models
for i in range(4):
    print('meter: '+ str(i))
    imprtc_df = pd.DataFrame()
    imprtc_df['feature'] = sample_train_X.drop('meter_reading', axis=1).columns   
    imprtc_df['importance'] = meter_models[i][0].feature_importances_
    imprtc_df.sort_values('importance', ascending=False, inplace= True)
    print(imprtc_df)


In [ ]:
# %%time
# ## Single fit single model

# gbm = LGBMRegressor(**gbm_params)
# f_train_X, f_train_y = getInFoldXY(train.index)
# gbm.fit(f_train_X, f_train_y)

In [ ]:
# Generate test_X
test_X = x_pipes.transform(
    test
        .merge(building_pipes.transform(building), on='building_id', how='left').drop(['row_id'], axis=1)
        .merge(weather_pipes.transform(weather_test), on=['site_id', 'timestamp'], how='left')
    )

print(test_X.sample(n=20,  random_state=42))
print(test_X.shape)
print(test_X.dtypes)

In [ ]:
def predMeters(test_X):
    test_y = test_X[['meter']] 
    test_y['meter_reading'] = np.nan
    for i in range(4):
        X = test_X[test_X['meter'] == i]
        if X.shape[0] > 0:
            preds = np.expm1(sum([model.predict(X) for model in meter_models[i]])/folds)
            test_y.loc[test_y['meter'] == i, 'meter_reading'] = preds
    return test_y['meter_reading'].tolist()
    
print(predMeters(test_X.sample(n=20,  random_state=42)))    


In [ ]:
# Predict using cross val models ensemble 
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
    res.append(predMeters(test_X.iloc[i:i+step_size]))
    i+=step_size
    gc.collect()


In [ ]:
# Save using cross val models ensemble 
res = np.concatenate(res)
print(len(res))
submission = pd.read_csv('../input/ashrae-energy-prediction/sample_submission.csv')
submission['meter_reading'] = res
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('submission_meter.csv.zip', index=False)
submission.shape

In [ ]:
# # Predict single model fit
# i=0
# res=[]
# step_size = 50000
# for j in tqdm(range(int(np.ceil(test_X.shape[0]/50000)))):
#    #res.append(np.expm1(sum([model.predict(test_X.iloc[i:i+step_size]) for model in models])/folds))
#    res.append(np.expm1(gbm.predict(test_X.iloc[i:i+step_size])))
#    i+=step_size
    